## Querying databases
We've already seen a couple of SQL queries when creating a database table and checking if data was present. Now we will go beyond those basics queries to do slightly more advanced queries, like searching and filtering.

In [1]:
# Work with an in-memory SQLite database again
import sqlite3
connection = sqlite3.connect(':memory:')

In [2]:
# Create a table again for holding a path and size, just like before
table = 'CREATE TABLE files (id integer primary key, path TEXT, bytes INTEGER)'
cursor = connection.cursor()
cursor.execute(table)
connection.commit()

There is a _large_files.py_ file that has a `files` variable which holds a list of tuples with some sample data we can use to populate the database. Import that module and use the list to iterate over it and then populate the database

In this section you will use a special SQL syntax in SQLite to insert values from Python into the SQL query.

In [5]:
from large_files import files

for metadata in files:
    query = 'INSERT INTO files(path, bytes) VALUES(?, ?)'
    # the execute() method accepts a query and optionally a tuple with values 
    # corresponding to the question marks in VALUES
    cursor.execute(query, metadata)
    connection.commit()


So far we've seen `CREATE` and `INSERT`. Let's try a new SQL statement to make a selection. The `SELECT` statement produces a result from one or more tables and from one or many rows. 

Note the particular (and strict) order of SQL statements:

- `SELECT`
- `FROM`
- `WHERE`
- `GROUP BY`
- `HAVING`
- `ORDER BY`

Since SQLite returns an iterator as a result always, then it is required to loop over the resulting object. Create a query to count the items in the `files` table. This query will use the `COUNT()` function that produces a number:

In [6]:
query = 'SELECT COUNT(id) from files'

for i in cursor.execute(query):
    print(i)

(13825,)


## Extracting distinct row data
Counting items is a good way of checking the amount of items that exist in the database. Without using `COUNT(id)` the previous query would've produced two thousand entries. Use the `LIMIT` statement to set the maximum number of entries that can be produced, then remove the `COUNT(id)` function and use `*` instead to use all rows

In [7]:
query = 'SELECT * from files LIMIT(10)'
for i in cursor.execute(query):
    print(i)

(1, '/Users/pandi/source/DataEngineering/file.txt', 6055)
(2, '/Users/pandi/source/DataEngineering/.DS_Store', 8196)
(3, '/Users/pandi/source/DataEngineering/README.md', 145)
(4, '/Users/pandi/source/DataEngineering/.gitignore', 19)
(5, '/Users/pandi/source/DataEngineering/test-sqllite3.db', 0)
(6, '/Users/pandi/source/DataEngineering/DataEngineeringEssentials/py_env_setup.txt', 1529)
(7, '/Users/pandi/source/DataEngineering/DataEngineeringEssentials/.DS_Store', 10244)
(8, '/Users/pandi/source/DataEngineering/DataEngineeringEssentials/requirements.txt', 833)
(9, '/Users/pandi/source/DataEngineering/DataEngineeringEssentials/ReadMe.md', 1593)
(10, '/Users/pandi/source/DataEngineering/DataEngineeringEssentials/Python_and_Pandas_4_DE/.DS_Store', 6148)


Using `*` means every row in the table. The table in this case is `files`. The next query specifies using the `id` row only. 

**Exercise:** Update the cell contents so that it shows paths instead.

In [8]:
query = 'SELECT id FROM files LIMIT(10)'
for i in cursor.execute(query):
    print(i)

(1,)
(2,)
(3,)
(4,)
(5,)
(6,)
(7,)
(8,)
(9,)
(10,)


The next query uses ID and Bytes. Update the query once again to select two rows in the table: `bytes` and `path`.

In [9]:
query = 'SELECT id,bytes FROM files LIMIT(10)'
for i in cursor.execute(query):
    print(i)

(1, 6055)
(2, 8196)
(3, 145)
(4, 19)
(5, 0)
(6, 1529)
(7, 10244)
(8, 833)
(9, 1593)
(10, 6148)


## Extracting distinct data
You now know how to extract data from certain rows and how to limit that data. Next, we'll use more SQL statements to further find and filter out results so that you can get specific results.

**Exercise:** Use the next query to find 10 files that are bigger than 1mb (1000000 bytes) using a new statement (`WHERE`):

In [10]:
query = 'SELECT path FROM files WHERE bytes>1000000 LIMIT(10)'
for i in cursor.execute(query):
    print(i)

('/Users/pandi/source/DataEngineering/DataEngineeringEssentials/Python_and_SQL_4_DE/sample_data/wine-ratings.csv',)
('/Users/pandi/source/DataEngineering/DataEngineeringEssentials/.venv/lib/python3.13/site-packages/debugpy/_vendored/pydevd/_pydevd_sys_monitoring/_pydevd_sys_monitoring_cython.c',)
('/Users/pandi/source/DataEngineering/DataEngineeringEssentials/.venv/lib/python3.13/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_cython.cpython-313-darwin.so',)
('/Users/pandi/source/DataEngineering/DataEngineeringEssentials/.venv/lib/python3.13/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_cython.c',)
('/Users/pandi/source/DataEngineering/DataEngineeringEssentials/.venv/lib/python3.13/site-packages/debugpy/_vendored/pydevd/_pydevd_frame_eval/pydevd_frame_evaluator.c',)
('/Users/pandi/source/DataEngineering/DataEngineeringEssentials/.venv/lib/python3.13/site-packages/numpy/_core/_multiarray_umath.cpython-313-darwin.so',)
('/Users/pandi/source/DataEngineering/Dat

The query shows the paths but not the sizes. 

**Exercise:** Try updating the previous query to show both the path and the size. 

In [11]:
query = ''
for i in cursor.execute(query):
    print(i)

SQL has many helper functions, in this case the next query uses `MAX()` which can find the highest value in a column. Do you think that `LIMIT(10)` makes sense in this query? Why? What happens if you remove the `LIMIT(10)` clause?

**Exercise:** Remote the `LIMIT()` clause and check your results

In [12]:
query = 'SELECT path,MAX(bytes) FROM files LIMIT(10)'
for i in cursor.execute(query):
    print(i)

('/Users/pandi/source/DataEngineering/DataEngineeringEssentials/Python_and_SQL_4_DE/sample_data/wine-ratings.csv', 13518834)


SQL queries can be compounded for more conditionals. In Python, you can make the query more readable by using triple quotes and adding the queries in a multi-line variable.

**Exercise:** Use other conditions to match different sizes and limit to a different number of entries returned

In [13]:
query = """
SELECT path,bytes FROM files 
    WHERE bytes>3000000 
    AND bytes<4400000 LIMIT(100)
"""
for i in cursor.execute(query):
    print(i)

('/Users/pandi/source/DataEngineering/DataEngineeringEssentials/.venv/lib/python3.13/site-packages/numpy/_core/_multiarray_umath.cpython-313-darwin.so', 3345784)
('/Users/pandi/source/DataEngineering/DataEngineeringEssentials/.venv/lib/python3.13/site-packages/zmq/.dylibs/libzmq.5.dylib', 3329056)
('/Users/pandi/source/DataEngineering/.git/objects/43/9af2970bfc159a6e2deb5c49a2099078c7a058', 3906409)


## Searching
Sometimes you can't tell exactly what is it that you are looking for in a query. SQL allows for matching patterns. In the file paths situation, you might know that a specific file ends with `.zip` but you don't know where it is. 

**Exercise:** Use the `LIKE` operator to match and find a cache file related to an Address Book application.

In [14]:
query = """
SELECT path,bytes FROM files 
    WHERE path LIKE '%AddressBook%'
"""
for i in cursor.execute(query):
    print(i)

Using `%` means to match any text of zero or more characters. So `%AddressBook%` is very lenient for anything before it and after it. Try adding a condition that filters the result by size. Anything over 2MB (or 2000000 bytes) and see if you can reduce the amount of output.

There are other variations for search like using an underscore (`_`). That means any single character. If you know that a file prefix or suffix is, you could use this to fine-tune your search.

**Exercise:** Use other search items for the paths found in your filesystem and try to match them